<a href="https://colab.research.google.com/github/VENUGOPALADA/SecondRepo/blob/main/Drones_And_Bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install gradio

In [12]:
pip install tensorflow

In [33]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import gradio as gr
import zipfile

dataset_path = '/content/D_A_B_C.zip'
extracted_folder_path = '/content/D_A_B_C_Dataset'

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

dataset_files = os.listdir(extracted_folder_path)
print("Extracted files:", dataset_files)

Extracted files: ['D_A_B']


In [34]:
data_folder_path = os.path.join(extracted_folder_path, 'D_A_B')  # Updated folder name
data_files = os.listdir(data_folder_path)

print("Contents of the dataset folder:", data_files)

Contents of the dataset folder: ['Test', 'Train']


In [35]:

train_path = os.path.join(data_folder_path, 'Train')
test_path = os.path.join(data_folder_path, 'Test')

train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

print("Training files:", train_files)
print("Testing files:", test_files)


Training files: ['birds', 'drones']
Testing files: ['birds', 'drones']


In [36]:
Model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # Drones, Birds
])

Model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1.0/255.0, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_path,
    batch_size=32,
    class_mode='categorical',
    target_size=(150, 150)
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    batch_size=32,
    class_mode='categorical',
    target_size=(150, 150)
)


Found 30 images belonging to 2 classes.
Found 326 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
history = Model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=5,
    validation_data=test_generator,
    validation_steps=test_generator.samples // 32,
    verbose=2
)



Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 - 8s - 8s/step - accuracy: 0.5667 - loss: 0.6940 - val_accuracy: 0.4969 - val_loss: 0.7967
Epoch 2/5


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1/1 - 5s - 5s/step - accuracy: 0.5000 - loss: 0.6973 - val_accuracy: 0.1667 - val_loss: 9.0756
Epoch 3/5
1/1 - 6s - 6s/step - accuracy: 0.5000 - loss: 5.2131 - val_accuracy: 0.5094 - val_loss: 0.9388
Epoch 4/5
1/1 - 6s - 6s/step - accuracy: 0.5000 - loss: 0.8664 - val_accuracy: 0.1667 - val_loss: 0.8918
Epoch 5/5
1/1 - 6s - 6s/step - accuracy: 0.5000 - loss: 0.7056 - val_accuracy: 0.5156 - val_loss: 0.6955


In [38]:
def classify_image(img):
    img = img.resize((150, 150))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = img_arr / 255.0

    pred = Model.predict(img_arr)

    labels = ['Drone', 'Bird']  # Updated labels
    pred_class = labels[np.argmax(pred)]

    return f"Predicted Class: {pred_class}"

In [39]:

# Gradio interface
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Text(),
    live=True,
    title="Drone vs Bird Classifier",
    description="Upload an image, and the model will classify it as 'Drone' or 'Bird'."
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9702560a821e573d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
